# Automatic Differentiation

Here, we will look at implementing automatic differentiation from scratch.

We will build a computational graph structure, similar to what is utilized in tensorflow and pytorch. This will only involve numpy and base python.

## Computational Graph

Our computational graph will be implemented through the use of global variables (this was how tensorflow 1.x operated).

In [1]:
import numpy as np

### 1. Graph Class

In [2]:
class Graph():
    
    def __init__(self):
        self.operators = set()
        self.constants = set()
        self.variables = set()
        self.placeholders = set()
        
        # NOTE: our notation for global variables will be an underscore
        global _g
        # we are initializing our graph object as a global instance
        _g = self
        
    def reset_counts(self, root):
        if hasattr(root, 'count'):
            root.count = 0
        else:
            for child in root.__subclasses__():
                self.reset_counts(child)
    
    def reset_session(self):
        try:
            del _g
        except:
            pass
        self.reset_counts(Node)
        
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.reset_session()

### 2. Node Class

In [3]:
class Node:
    def __init__(self):
        pass

### 3. Placeholder class

In [4]:
class Placeholder(Node):
    # placeholder holds a node and awaits further inputs at computation
    
    count = 0
    def __init__(self, name, dtype=float):
        _g.placeholders.add(self)
        self.value = None
        self.gradient = None
        self.name = f'Plc/{Placeholder.count}' if name is None else name
        Placeholder.count += 1
        
    def __repr__(self):
        return f'Placeholder: name:{self.name}, value:{self.value}'


### 4. Constant class

In [5]:
class Constant(Node):
    # constant node in computational graph
    count = 0
    def __init__(self, value, name=None):
        _g.constants.add(self)
        self._value = value
        self.gradient = None
        self.name = f'Const/{Constant.count}' if name is None else name
        Constant.count += 1
        
    def __repr__(self):
        return f'Constant: name:{self.name}, value:{self.value}'
    
    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self):
        raise VallueError('Cannot reassign constant')
        self.value = None
        self.gradient = None
        self.name = f'Plc/{Placeholder.count}' if name is None else name
        Placeholder.count += 1
    
    def __repr__(self):
        return f'Placeholder: name:{self.name}, value:{self.value}'
    

### 5. Variable Class

In [6]:
class Variable(Node):
    # variable node, we track these for differentiation during graph computation
    
    count = 0
    def __init__(self, value, name=None):
        _g.variables.add(self)
        self.value = value
        self.gradient = None
        self.name = f'Var/{Variable.count}' if name is None else name
        Variable.count += 1
    
    def __repr__(self):
        return f'Variable: name:{self.name}, value:{self.value}'

### 6. Operator Class

In [7]:
class Operator(Node):
    # An operator node in the computational graph.
    
    def __init__(self, name='Operator'):
        _g.operators.add(self)
        self.value = None
        self.inputs = []
        self.gradient = None
        self.name = name
    
    def __repr__(self):
        return f"Operator: name:{self.name}"

### Operators

We will need operators to do the actual operations in our computational graph. We will define basic operators here, and they will subclass the operators node.

In [8]:
class add(Operator):
    count = 0
    # binary addition
    def __init__(self, a, b, name=None):
        super().__init__(name)
        self.inputs=[a,b]
        self.name = f'add/{add.count}' if name is None else name
        add.count += 1
    
    def forward(self, a, b):
        return a + b
    
    def backward(self, a, b, dout):
        return dout, dout
    
class multiply(Operator):
    count = 0
    # binary multiplication
    def __init__(self, a, b, name=None):
        super().__init__(name)
        self.inputs = [a,b]
        self.name = f'mul/{multiply.count}' if name is None else name
        multiply.count += 1
    
    def forward(self, a, b):
        return a * b
    
    def backward(self, a, b, dout):
        return dout*b, dout*a
    
class divide(Operator):
    count = 0
    # binary division
    def __init__(self, a, b, name=None):
        super().__init__(name)
        self.inputs = [a,b]
        self.name = f'div/{divide.count}' if name is None else name
        divide.count += 1
        
    def forward(self, a, b):
        return a / b
    
    def backward(self, a, b, dout):
        return dout / b, dout * a / np.power(b,2)
    
class power(Operator):
    count = 0
    # raising to an exponent
    def __init__(self, a, b, name=None):
        super().__init__(name)
        self.inputs = [a,b]
        self.name = f'pow/{power.count}' if name is None else name
        power.count += 1
        
    def forward(self, a, b):
        return np.power(a, b)
    
    def backward(self, a, b, dout):
        return dout * b * np.power(a, b-1), dout * np.log(a) * np.power(a, b)
    
class matmul(Operator):
    count = 0
    # matrix multiplication, defaults to binary multiplication
    
    def __init__(self, a, b, name=None):
        super().__init__(name)
        self.inputs = [a, b]
        self.name = f'matmul/{matmul.count}' if name is None else name
        matmul.count += 1
        
    def forward(self, a, b):
        return a @ b
    
    def backward(self, a, b, dout):
        return dout @ b.T, a.T @ dout

### Overload Operators Above

We will now overload the operators defined above.

In [9]:
# this function will ensure that the operators we are comparing are graph nodes

def node_wrapper(func, self, other):
    if isinstance(other, Node):
        return func(self, other)
    if isinstance(other, float) or isinstance(other, int):
        return func(self, Constant(other))
    raise TypeError('Incompatible Types')

In [10]:
# overload these operators

Node.__add__ = lambda self, other: node_wrapper(add, self, other)
Node.__mul__ = lambda self, other: node_wrapper(multiply, self, other)
Node.__div__ = lambda self, other: node_wrapper(divide, self, other)
Node.__neg__ = lambda self: node_wrapper(multiply, self, Constant(-1))
Node.__pow__ = lambda self, other: node_wrapper(power, self, other)
Node.__matmul__ = lambda self, other: node_wrapper(matmul, self, other)


## Initialize a graph object

In [11]:
with Graph() as g:
    x = Variable(1.0)
    y = Variable(0.5)
    
    z = x*y + 5


In [12]:
print(g.constants)
print(g.variables)
print(g.operators)

{Placeholder: name:Const/0, value:5}
{Variable: name:Var/0, value:1.0, Variable: name:Var/1, value:0.5}
{Operator: name:add/0, Operator: name:mul/0}


## Autograd

We will now implement something similar to autograd from pytorch (or .apply_gradients from tensorflow).

In [13]:
def topological_sort(head_node=None, graph=_g):
    # do a topological sort of all nodes
    
    vis = set()
    ordering = []
    
    def _dfs(node):
        if node not in vis:
            vis.add(node)
            if isinstance(node, Operator):
                for input_node in node.inputs:
                    _dfs(input_node)
            ordering.append(node)
            
    if head_node is None:
        for node in graph.operators:
            _dfs(node)
    else:
        _dfs(head_node)
    
    return ordering

In [14]:
def forward_pass(order, feed_dict={}):
    
    for node in order:
        if isinstance(node, Placeholder):
            node.value = feed_dict[node.name]
        
        elif isinstance(node, Operator):
            node.value = node.forward(*[prev_node.value for prev_node in node.inputs])
            
    return order[-1].value

In [15]:
def backward_pass(order, target_node=None):
    
    vis = set()
    order[-1].gradient = 1
    for node in reversed(order):
        if isinstance(node, Operator):
            inputs = node.inputs
            grads = node.backward(*[x.value for x in inputs], dout=node.gradient)
            for inp, grad in zip(inputs, grads):
                if inp not in vis:
                    inp.gradient = grad
                else:
                    inp.gradient += grad
                vis.add(inp)
    return [node.gradient for node in order]

### Test out Feedforward

We will be using the following function:

$$f(x) = c_{1}(xy + c_{1}) + x$$

We expect that the gradients will be:

$$\nabla f = \begin{pmatrix} \frac{\partial f}{\partial x} \\[3pt] \frac{\partial f}{\partial y} \\[3pt] \frac{\partial f}{\partial c_{1}} \end{pmatrix} = \begin{pmatrix} c_{1}y + 1 \\[3pt] c_{1}x \\[3pt] 2c_{1} + xy \end{pmatrix}$$

In [16]:
val1, val2, val3 = 0.5, 0.6, 1.3

with Graph() as g:
    x = Variable(val1, name='x')
    y = Variable(val2, name='y')
    c1 = Constant(val3, name='c1')
    
    z = c1*(x*y + c1) + x
    
    # apply the topological sort to the outcome, then apply forward pass, backward pass to compute gradient
    order = topological_sort(z)
    res = forward_pass(order)
    grads = backward_pass(order)
    
    print('Node Order:')
    for node in order:
        print(node)
        
    print('-'*10)
    print(f'Forward pass expected: {(val1*val2 + val3)*val3 + val1}')
    print(f'Forward pass computed: {res}')

Node Order:
Placeholder: name:c1, value:1.3
Variable: name:x, value:0.5
Variable: name:y, value:0.6
Operator: name:mul/0
Operator: name:add/0
Operator: name:mul/1
Operator: name:add/1
----------
Forward pass expected: 2.58
Forward pass computed: 2.58


### Test out Gradient

In [17]:
dz_dx = [a for a in order if a.name=='x'][0]
dz_dy = [a for a in order if a.name=='y'][0]
dz_dc = [a for a in order if a.name=='c1'][0]

print(f"dz/dx expected = {val3*val2+1}")
print(f"dz/dx computed = {dz_dx.gradient}")

print(f"dz/dy expected = {val1*val3}")
print(f"dz/dy computed = {dz_dy.gradient}")

print(f"dz/dc expected = {val1*val2+2*val3}")
print(f"dz/dc computed = {dz_dc.gradient}")

dz/dx expected = 1.78
dz/dx computed = 1.78
dz/dy expected = 0.65
dz/dy computed = 0.65
dz/dc expected = 2.9
dz/dc computed = 2.9000000000000004


So our gradients work as expected (with a little bit of round off error). Let's see if this works on arbitrary tensors:

In [18]:
T = np.ones((5,5))
U = np.ones((5,5))
c = 6.0
with Graph() as g:
    x = Variable(T, name='T')
    y = Variable(U, name='U')
    z = Constant(6.0, name='c')
    
    # define function to differentiate
    h = x**3
    
    order2 = topological_sort(h)
    result2 = forward_pass(order2)
    gradients2 = backward_pass(order2)
    
    print('Node Order:')
    for node in order2:
        print(node)
        
    print('-'*10)
    print(f'Forward pass expected: {T**3}')
    print(f'Forward pass computed: {result2}')

Node Order:
Variable: name:T, value:[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
Placeholder: name:Const/1, value:3
Operator: name:pow/0
----------
Forward pass expected: [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
Forward pass computed: [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [19]:
dh_dT = [a for a in order2 if a.name == 'T'][0]

print(f'dh_dT expected = {3*(T**2)}')
print(f'dh_dT computed = {dh_dT.gradient}')

dh_dT expected = [[3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]]
dh_dT computed = [[3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]]


So this automatic gradient works for polynomial-like functions of arbitrary tensors!

One thing we need to work on is that we should define derivatives for functions like $\sin$, $\cos$, $\exp$, $\log$ so that we can cover all bases for elementary functions.